In [1]:
from huggingface_hub import login

login("Your API Key")

/home/quoppo/sglang1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:


import sglang as sgl
import json

#Choose one model to evaluate

gen = "l3cube-pune/marathi-gpt-gemma-2b"

# Initialize the generation engine.
gen_llm = sgl.Engine(model_path=gen)
sampling_params = {"temperature": 0.5, "top_p": 0.95}

# Define 20 questions/instructions.
questions = [
    # Direct Questions in Hindi
    "नमस्ते! अपने पसंदीदा त्योहार के बारे में बताइए।",
    "आज के भारत में शिक्षा व्यवस्था की स्थिति पर आप क्या सोचते हैं?",
    "भारतीय संस्कृति में परिवार का महत्व क्या है?",
    "आपके विचार में भारतीय राजनीति में सुधार की आवश्यकता क्यों है?",

    # Direct Questions in Tamil
    "தமிழில் உங்கள் நாட்டின் பரம்பரை உணர்வுகளை விவரிக்கவும்.",
    "இந்திய உணவின் சிறப்புகளை விளக்கவும்.",
    "தமிழில், உங்களுக்கு பிடித்த ஆன்மீக பாடத்தை பற்றி பேசுங்கள்.",
    "இந்திய திரை உலகத்தில் நீங்கள் விரும்பும் நடிகர் அல்லது நடிகை யார்?",

    # Direct Questions in Bengali
    "বাংলায় আপনার দেশের ঐতিহ্য ও সংস্কৃতি নিয়ে আলোচনা করুন।",
    "আপনার মতে, ভারতে কোন খেলাটি সবচেয়ে জনপ্রিয় এবং কেন?",
    "বাংলায়, আপনার প্রিয় কবিতাটি লিখুন।",
    "ভারতের কোন ঐতিহাসিক ঘটনা আপনার উপর গভীর প্রভাব ফেলেছে?",

    # Direct Questions in Telugu
    "మీరు తెలుగులో భారతదేశంలో విద్యా వ్యవస్థ గురించి మీ అభిప్రాయాన్ని చెప్పండి.",
    "భారతీయ సాంప్రదాయ వంటకాల గురించి వివరించండి.",
    "తెలుగులో, మీరు ఇష్టపడే సాంస్కృతిక కార్యక్రమం గురించి వివరించండి.",
    "భారతదేశంలో రాజకీయ పరిస్థితులపై మీ అభిప్రాయాన్ని తెలుగులో వ్యక్తపరచండి.",

    # Direct Questions in Malayalam
    "നിങ്ങളുടെ നാട്ടിലെ വിദ്യാഭ്യാസ സംവിധാനത്തെക്കുറിച്ച് മലയാളത്തിൽ വിശദീകരിക്കുക.",
    "കേരളത്തിലെ സംസ്‌കാരിക ചടങ്ങുകളെപ്പറ്റി പറയൂ.",
    "മലയാളത്തിൽ, നിങ്ങളെ ഏറ്റവും സന്തോഷിപ്പിക്കുന്ന ഒരു കഥ പങ്കുവെക്കൂ.",
    "കേരളത്തിലെ പരിസ്ഥിതി പ്രശ്നങ്ങളെക്കുറിച്ച് നിങ്ങളുടെ അഭിപ്രായം എന്താണ്?",

    # Direct Questions in Marathi
    "तुमच्या मते भारतातील सध्याच्या आर्थिक परिस्थितीबद्दल काय मत आहे?",
    "भारतीय सणांबद्दल मराठीत लिहा आणि त्यांचा सांस्कृतिक महत्त्व स्पष्ट करा.",
    "मराठीत, तुमचा आवडता कवी कोण आहे आणि का?",
    "भारतातील सामाजिक बदलांवर तुमचे मत काय आहे?",

    # Direct Questions in English
    "Write a short poem about a soldier in English.",
    "Explain the process of photosynthesis in simple terms.",
    "Describe the importance of cybersecurity in modern society.",
    "What are the benefits of regular exercise?",
    "Outline the major causes of climate change.",
    "Discuss the impact of social media on communication.",
    "Explain the significance of the scientific method.",
    "Compare two different types of renewable energy sources.",

    # Mixed Language / Multilingual Prompts
    "Explain how a computer works in layman’s terms, then explain the same concept in Hindi.",
    "Translate the following sentence into Tamil: 'The future of technology is bright and full of promise.'",
    "What are some popular proverbs in Indian languages? Provide at least three and explain their meanings in English.",
    "Simulate a conversation in Hindi about your favorite Indian festival. Then, summarize the conversation in English.",

    # Additional Direct Questions in Indian Languages
    "हिन्दी में बताइए कि भारत में कोरोना महामारी के बाद क्या बदलाव आया है?",
    "தமிழில், கடந்தாண்டு நடந்த முக்கிய நிகழ்வுகளை பற்றி விவரிக்கவும்.",
    "বাংলায়, সাম্প্রতিক সময়ে দেশের সামাজিক পরিবর্তন নিয়ে আপনার অভিমত কী?",
    "తెలుగులో, సమకాలీన భారతీయ సాహిత్యం గురించి మీ అభిప్రాయాన్ని చెప్పండి.",
    "മലയാളത്തിൽ, സമകാലീന ഇന്ത്യൻ സിനിമയെക്കുറിച്ച് നിങ്ങളുടെ അഭിപ്രായം എന്താണ്?",
    "मराठीत, कोरोना काळातील शैक्षणिक बदलांवर तुमचे विचार स्पष्ट करा."
]

responses = []

# Generate and store responses.
for question in questions:
    response = gen_llm.generate(question, sampling_params, stream=False)
    answer = response.get('text', '').strip()
    responses.append({
        "instruction": question,
        "answer": answer
    })
    print(f"Processed: {question}")
file_name = f"genMLE{gen.replace('/', '-')}.json"
# Save the generated responses to a JSON file.
with open(file_name ,"w") as f:
    json.dump(responses, f, indent=4)

print(f"Generation complete. Responses stored in {file_name}.json")
# this will output the json file by model name which can be evaluated manualy or by other llm

/home/quoppo/sglang1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-06 13:57:18,792	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:03<00:03,  3.08s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:06<00:00,  3.14s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:06<00:00,  3.13s/it]

100%|██████████| 4/4 [00:01<00:00,  3.44it/s]


Processed: नमस्ते! अपने पसंदीदा त्योहार के बारे में बताइए।
Processed: आज के भारत में शिक्षा व्यवस्था की स्थिति पर आप क्या सोचते हैं?
Processed: भारतीय संस्कृति में परिवार का महत्व क्या है?
Processed: आपके विचार में भारतीय राजनीति में सुधार की आवश्यकता क्यों है?
Processed: தமிழில் உங்கள் நாட்டின் பரம்பரை உணர்வுகளை விவரிக்கவும்.
Processed: இந்திய உணவின் சிறப்புகளை விளக்கவும்.
Processed: தமிழில், உங்களுக்கு பிடித்த ஆன்மீக பாடத்தை பற்றி பேசுங்கள்.
Processed: இந்திய திரை உலகத்தில் நீங்கள் விரும்பும் நடிகர் அல்லது நடிகை யார்?
Processed: বাংলায় আপনার দেশের ঐতিহ্য ও সংস্কৃতি নিয়ে আলোচনা করুন।
Processed: আপনার মতে, ভারতে কোন খেলাটি সবচেয়ে জনপ্রিয় এবং কেন?
Processed: বাংলায়, আপনার প্রিয় কবিতাটি লিখুন।
Processed: ভারতের কোন ঐতিহাসিক ঘটনা আপনার উপর গভীর প্রভাব ফেলেছে?
Processed: మీరు తెలుగులో భారతదేశంలో విద్యా వ్యవస్థ గురించి మీ అభిప్రాయాన్ని చెప్పండి.
Processed: భారతీయ సాంప్రదాయ వంటకాల గురించి వివరించండి.
Processed: తెలుగులో, మీరు ఇష్టపడే సాంస్కృతిక కార్యక్రమం గురించి వివరించండి.
Processed:

In [ ]:
# this will output the json file by model name which can be evaluated manualy or by other llm

In [1]:
#Evaluatio with Transformers library 

import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the model name and set the device (GPU if available)
model_name = "l3cube-pune/marathi-gpt-gemma-2b"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model, and move the model to the device
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.eval()  # Set the model to evaluation mode

# Sampling parameters
sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_length": 300,
    "do_sample": True  # Enable sampling for diversity in responses
}


questions = [
    # Direct Questions in Hindi
    "नमस्ते! अपने पसंदीदा त्योहार के बारे में बताइए।",
    "आज के भारत में शिक्षा व्यवस्था की स्थिति पर आप क्या सोचते हैं?",
    "भारतीय संस्कृति में परिवार का महत्व क्या है?",
    "आपके विचार में भारतीय राजनीति में सुधार की आवश्यकता क्यों है?",

    # Direct Questions in Tamil
    "தமிழில் உங்கள் நாட்டின் பரம்பரை உணர்வுகளை விவரிக்கவும்.",
    "இந்திய உணவின் சிறப்புகளை விளக்கவும்.",
    "தமிழில், உங்களுக்கு பிடித்த ஆன்மீக பாடத்தை பற்றி பேசுங்கள்.",
    "இந்திய திரை உலகத்தில் நீங்கள் விரும்பும் நடிகர் அல்லது நடிகை யார்?",

    # Direct Questions in Bengali
    "বাংলায় আপনার দেশের ঐতিহ্য ও সংস্কৃতি নিয়ে আলোচনা করুন।",
    "আপনার মতে, ভারতে কোন খেলাটি সবচেয়ে জনপ্রিয় এবং কেন?",
    "বাংলায়, আপনার প্রিয় কবিতাটি লিখুন।",
    "ভারতের কোন ঐতিহাসিক ঘটনা আপনার উপর গভীর প্রভাব ফেলেছে?",

    # Direct Questions in Telugu
    "మీరు తెలుగులో భారతదేశంలో విద్యా వ్యవస్థ గురించి మీ అభిప్రాయాన్ని చెప్పండి.",
    "భారతీయ సాంప్రదాయ వంటకాల గురించి వివరించండి.",
    "తెలుగులో, మీరు ఇష్టపడే సాంస్కృతిక కార్యక్రమం గురించి వివరించండి.",
    "భారతదేశంలో రాజకీయ పరిస్థితులపై మీ అభిప్రాయాన్ని తెలుగులో వ్యక్తపరచండి.",

    # Direct Questions in Malayalam
    "നിങ്ങളുടെ നാട്ടിലെ വിദ്യാഭ്യാസ സംവിധാനത്തെക്കുറിച്ച് മലയാളത്തിൽ വിശദീകരിക്കുക.",
    "കേരളത്തിലെ സംസ്‌കാരിക ചടങ്ങുകളെപ്പറ്റി പറയൂ.",
    "മലയാളത്തിൽ, നിങ്ങളെ ഏറ്റവും സന്തോഷിപ്പിക്കുന്ന ഒരു കഥ പങ്കുവെക്കൂ.",
    "കേരളത്തിലെ പരിസ്ഥിതി പ്രശ്നങ്ങളെക്കുറിച്ച് നിങ്ങളുടെ അഭിപ്രായം എന്താണ്?",

    # Direct Questions in Marathi
    "तुमच्या मते भारतातील सध्याच्या आर्थिक परिस्थितीबद्दल काय मत आहे?",
    "भारतीय सणांबद्दल मराठीत लिहा आणि त्यांचा सांस्कृतिक महत्त्व स्पष्ट करा.",
    "मराठीत, तुमचा आवडता कवी कोण आहे आणि का?",
    "भारतातील सामाजिक बदलांवर तुमचे मत काय आहे?",

    # Direct Questions in English
    "Write a short poem about a soldier in English.",
    "Explain the process of photosynthesis in simple terms.",
    "Describe the importance of cybersecurity in modern society.",
    "What are the benefits of regular exercise?",
    "Outline the major causes of climate change.",
    "Discuss the impact of social media on communication.",
    "Explain the significance of the scientific method.",
    "Compare two different types of renewable energy sources.",

    # Mixed Language / Multilingual Prompts
    "Explain how a computer works in layman’s terms, then explain the same concept in Hindi.",
    "Translate the following sentence into Tamil: 'The future of technology is bright and full of promise.'",
    "What are some popular proverbs in Indian languages? Provide at least three and explain their meanings in English.",
    "Simulate a conversation in Hindi about your favorite Indian festival. Then, summarize the conversation in English.",

    # Additional Direct Questions in Indian Languages
    "हिन्दी में बताइए कि भारत में कोरोना महामारी के बाद क्या बदलाव आया है?",
    "தமிழில், கடந்தாண்டு நடந்த முக்கிய நிகழ்வுகளை பற்றி விவரிக்கவும்.",
    "বাংলায়, সাম্প্রতিক সময়ে দেশের সামাজিক পরিবর্তন নিয়ে আপনার অভিমত কী?",
    "తెలుగులో, సమకాలీన భారతీయ సాహిత్యం గురించి మీ అభిప్రాయాన్ని చెప్పండి.",
    "മലയാളത്തിൽ, സമകാലീന ഇന്ത്യൻ സിനിമയെക്കുറിച്ച് നിങ്ങളുടെ അഭിപ്രായം എന്താണ്?",
    "मराठीत, कोरोना काळातील शैक्षणिक बदलांवर तुमचे विचार स्पष्ट करा."
]
responses = []

# Generate and store responses
for question in questions:
    # Tokenize input and explicitly move each tensor to the same device as the model (i.e., CUDA)
    inputs = {k: v.to(device) for k, v in tokenizer(question, return_tensors="pt").items()}

    # Generate response
    output = model.generate(
        **inputs,
        temperature=sampling_params["temperature"],
        top_p=sampling_params["top_p"],
        max_length=sampling_params["max_length"],
        do_sample=sampling_params["do_sample"]
    )

    # Decode generated text
    response_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    responses.append({
        "instruction": question,
        "answer": response_text.strip()
    })

    print(f"Processed: {question}")

# Prepare a safe file name (replace "/" if present in model_name)
file_name = f"gen{model_name.replace('/', '-')}.json"

# Save the generated responses to a JSON file.
with open(file_name, "w") as f:
    json.dump(responses, f, indent=4)

print(f"Generation complete. Responses stored in {file_name}")

# this will output the json file by model name which can be evaluated manualy or by other llm

SyntaxError: invalid decimal literal (2577764809.py, line 7)